In [1]:
using Globtim
using CSV
using DataFrames
include("../src/lib_func.jl")

# Load the dataframe from the CSV file
df_2d = CSV.read("../data/camel_3_d6.csv", DataFrame)

# Constants and Parameters
d = 3      # Degree 
const n, a, b = 4, 5, 1 
const scale_factor = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .1 , 2 / 10  # Sampling parameters
# const tol_l2 = 1e-1               # Define the tolerance for the L2-norm
const tol_l2 = 1e-0
# The objective function
f = camel_3_by_3 # Objective function

camel_3_by_3 (generic function with 1 method)

In [2]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end

current L2-norm: 11.043955448105113
Number of samples: 7
current L2-norm: 1.2403628765940171
Number of samples: 9
attained the desired L2-norm: 0.6632549772537315


In [3]:
using DynamicPolynomials, HomotopyContinuation, ProgressLogging, DataFrames
@polyvar(x[1:n]) # Define polynomial ring 
ap = main_nd(n, d, poly_approx.coeffs)
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x. 
PolynomialApproximant = sum(Float64.(ap) .* MonomialVector(x, 0:d))
grad = differentiate.(PolynomialApproximant, x)
sys = System(grad)

System of length 4
 4 variables: x₁, x₂, x₃, x₄

 5.77223114470149e-13 - 2829.6875*x₁ + 24.9999999999999*x₂ - 3.72453149612998e-15*x₃ + 8.5090633651186e-14*x₄ + 2.92572465855782e-13*x₂*x₁ + 4.45977667269172e-14*x₂*x₁^2 + 2.45702392458055e-13*x₂*x₁^3 - 1.1753284907681e-14*x₂*x₃ - 1.36699030237833e-14*x₂*x₃^2 + 1.07866681840776e-14*x₂*x₃^3 + 2.87072908430546e-12*x₂^2*x₁ - 1.27098040061202e-13*x₂^2*x₁^2 - 1.0160002753919e-15*x₂^2*x₃ + 8.39284203692099e-14*x₂^2*x₃^2 - 8.25087391535879e-13*x₂^3*x₁ - 9.09366109586797e-16*x₂^3*x₃ - 8.5200517116761e-13*x₃*x₁ - 1.13497980804004e-13*x₃*x₁^2 - 2.42161324562692e-12*x₃*x₁^3 - 4.167419647848e-12*x₃^2*x₁ - 3.20351853259972e-13*x₃^2*x₁^2 + 5.3881321321408e-12*x₃^3*x₁ + 8.71343907442347e-13*x₄*x₁ - 1.57833666952053e-13*x₄*x₁^2 - 6.46926198538804e-12*x₄*x₁^3 + 3.11046103889712e-14*x₄*x₂ - 1.45721161769676e-14*x₄*x₂^2 + 7.7527979985767e-14*x₄*x₂^3 - 5.76957131070486e-14*x₄*x₃ - 2.80618503741507e-14*x₄*x₃^2 + 6.28592974022526e-14*x₄*x₃^3 - 5.5529617068762

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [4]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

Tracking 256 paths...   1%|▎                            |  ETA: 0:34:03



Tracking 256 paths...  98%|████████████████████████████▌|  ETA: 0:00:00



Tracking 256 paths... 100%|█████████████████████████████| Time: 0:00:17
  # paths tracked:                  256
  # non-singular solutions (real):  256 (32)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         256 (32)


32-element Vector{Vector{Float64}}:
 [-3.067022722077811, 14874.258144398214, 2.8059131021667176, -8930.87589930782]
 [-1.1108780878610012e13, -4.224637444555244e13, -7.0089891669144e13, 7.665244060006493e11]
 [-1.44648733878844e15, -3.5797202629692206e14, -3.1843841764107256e14, 4.8239073717796644e14]
 [-2.294495120706291e15, 6.5475613549742e14, 4.6249346812808306e14, -7.786081243524572e14]
 [3.081590821241988e-16, 9.414512480044321e-15, -0.4675786647103389, 0.23378933235511085]
 [1.259026563750613e13, 4.424046636910685e13, -7.095986899303712e13, 3.765443292143204e13]
 [-2.2939174301768325e12, -3.901139806057607e13, -6.7260499001253e13, 3.560987822678037e13]
 [-0.4675786647103433, 0.23378933235517138, -0.46757866471033915, 0.23378933235511853]
 [-2.866430388800613e15, -5.292522836576857e14, -5.986092922697139e14, -1.0638705220629731e15]
 [-0.46757866471034315, 0.23378933235515748, 2.148507232967405e-15, -7.358385857315489e-14]
 ⋮
 [1.3870756260537291e13, 4.398958283288187e13, -7.41683

Construct a Dataframe with only the critical points of the approximant $w_d$ which fall into the $[-1, 1]^4$ domain and rescale them to the original domain. 

In [5]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1 && -1 < point[3] < 1 && -1 < point[4] < 1 
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x1 = scale_factor * Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_x2 = scale_factor * Float64[point[2] for point in filtered_points] 
h_x3 = scale_factor * Float64[point[3] for point in filtered_points] 
h_x4 = scale_factor * Float64[point[4] for point in filtered_points] 

height = map(p -> f(p), zip(h_x1, h_x2, h_x3, h_x4)) # Compute the height of the critical points
df = DataFrame(x1 = h_x1, x2 = h_x2, x3 = h_x3, x4 = h_x4, height = height); # Create a DataFrame

Now we want to compute the distance between the critical points of the 2d function.

The true "exact" critical points of the 2d `Camel3` function are stored in `df_2d`. 

We compute which of those are the closest to the `[1:2]` and `[3:4]` coordinates of the critical points of our 4d approximant.  

The true critical points are stored in `df_2d`. Need to split to distance to minima first coordinate and second set of coordinates ten combine them.


In [6]:
total_distance = Float64[]
for i in 1:nrow(df)
    distances_1 = [norm([df.x1[i], df.x2[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    distances_2 = [norm([df.x3[i], df.x4[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    # Compute distances to all points in df_2d
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2) 

    # Extract the corresponding points
    point_1 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    point_2 = [df_2d.x[min_index_1], df_2d.y[min_index_1], df_2d.x[min_index_2], df_2d.y[min_index_2]]

    # Compute total distance
    tot_dist = norm(point_1 - point_2)
    push!(total_distance, tot_dist)

end
df.total_distance = total_distance
sorted_df = sort(df, :total_distance)


Row,x1,x2,x3,x4,height,total_distance
,Float64,Float64,Float64,Float64,Float64,Float64
1,9.79785e-16,-4.54516e-15,1.08709e-14,-3.71825e-13,1.34466e-25,3.16521e-13
2,1.5408e-15,4.70726e-14,-2.33789,1.16895,5.41125,0.735762
3,1.30145e-15,2.7828e-14,2.33789,-1.16895,5.41125,0.735762
4,-2.33789,1.16895,1.07425e-14,-3.67919e-13,5.41125,0.735762
5,2.33789,-1.16895,1.05709e-14,-3.78512e-13,5.41125,0.735762
6,-2.33789,1.16895,-2.33789,1.16895,10.8225,1.04053
7,2.33789,-1.16895,-2.33789,1.16895,10.8225,1.04053
8,-2.33789,1.16895,2.33789,-1.16895,10.8225,1.04053
9,2.33789,-1.16895,2.33789,-1.16895,10.8225,1.04053


In [7]:
using Optim
df.steps = zeros(nrow(df))
df.converged = zeros(nrow(df))

for i in 1:nrow(df)
    # println("Optimizing for point $i")
    x0 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    #  + 0.2 * randn(Float64, 4)
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    # df.dist_to_loc_min[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged
    # summary(res)
end
df

Row,x1,x2,x3,x4,height,total_distance,steps,converged
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.5408e-15,4.70726e-14,-2.33789,1.16895,5.41125,0.735762,8.0,1.0
2,-2.33789,1.16895,-2.33789,1.16895,10.8225,1.04053,8.0,1.0
3,-2.33789,1.16895,1.07425e-14,-3.67919e-13,5.41125,0.735762,8.0,1.0
4,2.33789,-1.16895,1.05709e-14,-3.78512e-13,5.41125,0.735762,8.0,1.0
5,2.33789,-1.16895,2.33789,-1.16895,10.8225,1.04053,8.0,1.0
6,9.79785e-16,-4.54516e-15,1.08709e-14,-3.71825e-13,1.34466e-25,3.16521e-13,0.0,1.0
7,1.30145e-15,2.7828e-14,2.33789,-1.16895,5.41125,0.735762,8.0,1.0
8,-2.33789,1.16895,2.33789,-1.16895,10.8225,1.04053,8.0,1.0
9,2.33789,-1.16895,-2.33789,1.16895,10.8225,1.04053,8.0,1.0
